# Línea base estaciones MiMacro

Generación de plots con la línea base de Mi Macro Periférico

In [1]:
import os
import sys
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aqiGDL
import numpy as np
%matplotlib inline

Colores y estilo de matplotlib a usar

In [2]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
colors = ['7A76C2', 'ff6e9c98', 'f62196', '18c0c4', 'f3907e', '66E9EC']

## Estaciones y red
El siguiente código carga las estaciones de Mi Macro a un GeoDataFrame y la red vial de Guadalajara

In [3]:
gdf_est = aqiGDL.gdf_from_db('estaciones_gdl','Estaciones')
gdf_est['codigo'] = gdf_est['Name'].apply(lambda x: x.split('.')[0])
gdf_est['nombre'] = gdf_est['Name'].apply(lambda x: x.split('. ')[1])
gdf_est = ox.project_gdf(gdf_est,to_crs='EPSG:32613')

In [4]:
G = ox.graph_from_bbox(20.751857,20.523110,-103.201328,-103.468643)
edges = ox.graph_to_gdfs(G, nodes=False)
edges = ox.project_gdf(edges,to_crs=gdf_est.crs)

## Carga estimaciones
La siguiente celda carga las estimaciones de los contaminantes para las estaciones de Mi Macro Periférico, estas estimaciones fueron creadas a partir de los datos disponibles por el Sistema Estatal de Monitoreo  SIMAJ. El cálculo se realizo por medio de interpolar los valores promedios de las estaciones SIMAJ en las estaciones de Mi Macro Periférico, en este momento no se tomaron en cuenta dinámicas ambientales como la disperción de contaminantes por los vientos.

In [5]:
inicio = '2013-21-31'
fin = '2019-12-31'
#query = f"SELECT * FROM data.mimacro_data_day WHERE \"FECHA\" between \'{inicio}\' and \'{fin}\'"
query = f"SELECT * FROM data.mimacro_data_week"
df = aqiGDL.df_from_query(query)
#df['FECHA'] = pd.to_datetime(df['FECHA']).dt.date
#df.sort_values(by=['FECHA'], inplace=True)
df.head(5)

,S_ID,S_YEAR,PARAM,EST_MIMACRO,CONC,DESV_EST,LONG,LAT
0,S1,S1-2014,PM10,10,46.044172,11.388455,-103.400806,20.606105
1,S1,S1-2014,PM10,18,34.317933,8.400739,-103.441604,20.644130
2,S1,S1-2014,PM10,19,35.698197,8.734699,-103.443536,20.648760
3,S1,S1-2014,PM10,1,85.862141,29.624801,-103.319783,20.591898
4,S1,S1-2014,PM10,20,37.633964,9.256137,-103.446802,20.656594


Red base de Guadalajara

In [6]:
df_week = df

In [12]:
errors = []
for est in df_week.EST_MIMACRO.unique().tolist():
    fig, axes = plt.subplots(2,3,figsize=(24,14), sharex=True)
    for param, ax, color in zip(df_week.PARAM.unique().tolist(), axes.flatten()[1:], colors):
        df_temp = df_week[(df_week['EST_MIMACRO']==est) & (df_week['PARAM']==param)]
        ax.plot(df_temp['S_YEAR'], df_temp['CONC'], label=param, color='#'+color)
        #rellena el espacio entre desviaciones estandar
        ax.fill_between(range(len(df_temp)), 
                               df_temp['DESV_EST']*-1+df_temp['CONC'], 
                                df_temp['DESV_EST']*1+df_temp['CONC'], 
                               facecolor='#9ddc9b', 
                               alpha=0.25)

        ax.set_title(param,fontsize=20)
        ax.tick_params(axis='x',labelrotation=45)
        x_ticks = np.arange(0, len(df_temp['S_YEAR']),15)
        for i in range(0,3):
            plt.sca(axes[1, i])
            plt.xticks(x_ticks, horizontalalignment="right");
            
    a00 = axes[0,0]
    shax = a00.get_shared_x_axes()
    shax.remove(a00)
    edges.plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.1, zorder=-1)
    edges[(edges['highway']=='primary') | (edges['highway']=='secondary')].plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.5, zorder=0)
    gdf_est.plot(ax=axes[0][0], color='k', alpha=0.85, zorder=1)
    gdf_est[gdf_est['codigo']==est].plot(ax=axes[0][0], color='#ba0d38', alpha=0.85, zorder=2, markersize=150)
    axes[0][0].axis('off')
    estacion = gdf_est[gdf_est['codigo']==est]['nombre'].values[0]
    fecha_1 = '2014-01-01'
    fecha_2 = '2019-12-31'
    fig.suptitle(f'Estación {estacion}\n{fecha_1} -- {fecha_2}', fontsize=30)
    plt.savefig(f'../output/figures/estmacro/{est}_{fecha_1}_{fecha_2}.png',dpi=300)
    plt.close()